Import the necessary libraries

In [29]:
import numpy as np
import pandas as pd

from joblib import load

import random
import psutil
import time

import warnings
warnings.filterwarnings('ignore')

Split the dataset into data packages, from which statistical parameters will be obtained

In [30]:
function_names = [
    'std', 'aad', 'mad', 'median', 'IQR'
]

functions = [
    lambda x: x.std(), 
    lambda x: np.mean(np.absolute(x - np.mean(x))),
    lambda x: np.median(np.absolute(x - np.median(x))),
    lambda x: np.median(x),
    lambda x: np.percentile(x, 75) - np.percentile(x, 25)
]

def processing(dataframe):
    new_dataframe = pd.DataFrame()
    for i, func in enumerate(functions):
        for col_name in dataframe.columns:
            new_name = f"{col_name}_{function_names[i]}"
            new_dataframe[new_name] = [func(dataframe[col_name])]
    return new_dataframe

def predict_label(df):

    fdf = processing(df)
    model = load('model.sav')
    prediction = np.argmax(model.predict(fdf), axis=1)
    
    return prediction

In [34]:
df = pd.read_csv('featured_data.csv')
df.head()

,label,ftop_acc_x,ftop_acc_y,ftop_acc_z,ftop_gyr_x,ftop_gyr_y,ftop_gyr_z,fdown_acc_x,fdown_acc_y,fdown_acc_z,fdown_gyr_x,fdown_gyr_y,fdown_gyr_z,dummy_label
0,Straight,0.077063,0.112141,0.058833,0.002225,0.038245,0.027952,-0.015480,-0.020803,-0.077544,-0.052433,0.137139,0.023294,0
1,Straight,0.074042,0.105587,0.056738,0.002044,0.032771,0.029252,-0.004156,-0.026806,-0.067851,-0.051849,0.077517,0.021766,0
2,Straight,0.069944,0.094662,0.060391,0.002226,0.025356,0.029219,0.006392,-0.028206,-0.055579,-0.050252,0.011438,0.018201,0
3,Straight,0.065650,0.083117,0.067682,0.003137,0.015944,0.026987,0.016057,-0.024883,-0.042477,-0.047516,-0.059940,0.012402,0
4,Straight,0.062471,0.074866,0.075738,0.005075,0.004790,0.022240,0.025384,-0.018131,-0.030027,-0.043691,-0.134249,0.004401,0


Algorithm for testing the model for memory and time costs

In [33]:
memory = []
ex_time = []
window_size = 120

for _ in range(100):
    start_index = random.randint(0, len(df) - window_size)
    test_df = df.iloc[start_index:start_index + window_size]
    y_test = test_df['dummy_label'].mode().values[0]

    process = psutil.Process()
    memory_before = process.memory_info().rss
    start_time = time.time()
    
    prediction = predict_label(test_df.drop(['label', 'dummy_label'], axis=1))

    end_time = time.time()
    memory_after = process.memory_info().rss
    
    execution_time = end_time - start_time
    ex_time.append(execution_time)
    memory_used = memory_after - memory_before
    memory.append(memory_used / 1024**2)
    
    print(f'Expected: {prediction[0]} Result: {y_test}')

print(f"Average execution time: {np.round(np.array(ex_time).mean(), 3)} seconds")
print(f"Average memory used by the function: {np.round(np.array(memory).mean(), 3)} Mb")    

1/1 [==============================] - 0s 50ms/step
Expected: 0 Result: 0
1/1 [==============================] - 0s 53ms/step
Expected: 2 Result: 2
1/1 [==============================] - 0s 64ms/step
Expected: 1 Result: 1
1/1 [==============================] - 0s 59ms/step
Expected: 1 Result: 1
1/1 [==============================] - 0s 53ms/step
Expected: 4 Result: 4
1/1 [==============================] - 0s 51ms/step
Expected: 4 Result: 4
1/1 [==============================] - 0s 50ms/step
Expected: 3 Result: 3
1/1 [==============================] - 0s 50ms/step
Expected: 4 Result: 4
1/1 [==============================] - 0s 50ms/step
Expected: 4 Result: 4
1/1 [==============================] - 0s 48ms/step
Expected: 0 Result: 0
1/1 [==============================] - 0s 48ms/step
Expected: 0 Result: 0
1/1 [==============================] - 0s 48ms/step
Expected: 3 Result: 3
1/1 [==============================] - 0s 56ms/step
Expected: 0 Result: 0
1/1 [==============================] -